<a href="https://colab.research.google.com/github/gilzalc/Data-mining/blob/main/ex1Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 1- Web scraping

###  Matan Chen & Gil Zalcberg

In [ ]:
import matplotlib 
import matplotlib.pyplot as plt
import requests 
from bs4 import BeautifulSoup 
import time
import re
import codecs

In [ ]:
url = 'https://www.imdb.com/'
r = requests.get(url)
r

<Response [200]>

In [ ]:
r = requests.get(url, timeout = 2)
r

<Response [200]>

In [ ]:
r.url

'https://www.imdb.com/'

In [ ]:
from warnings import warn
url = 'https://www.imdb.com/'
r = requests.get(url)
print(r)
if r.status_code != 200:
    warn('Request: {}; Status code: {}'.format(requests, r.status_code))


<Response [200]>


## Web scraping IMDB's time travel movies section


In [ ]:
url = "https://www.imdb.com/title/tt12361974/"
source_code = requests.get(url, timeout = 5)
print(source_code.status_code)

plain_text = source_code.text
soup = BeautifulSoup(plain_text)
print(soup.prettify())

200
<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///title/tt12361974?src=mdot" name="apple-itunes-app"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   Zack Snyder's Justice League (2021) - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld

### Class: Movie

In [ ]:
class Movie:
    def __init__(self,name,year,pgRating,length,gen,director,stars,IMDBrate,votes,gross,text):
        self._Name = name
        self._Year = year
        self._pgRating = pgRating
        self._Length = length
        self._Genres = gen
        self._Director = director
        self._Stars = stars
        self._IMDBRating = IMDBrate
        self._NumberOfVotes = votes
        self._Gross = gross
        self._FullText = text
#     def __str__(self):
#         return self._Name+"\n"

    

### Page parser function

In [ ]:
# import re
# from bs4 import NavigableString
def ParseUrlToMovie(url):
    source_code = requests.get(url, timeout = 5)
    print(source_code.status_code)

    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)

    soup_name = soup.find('div', "title_bar_wrapper").find('h1')
#     soup.find('div', class_ = "title_wrapper").find('h1', class_="")
            
    Name = soup_name.contents[0].strip()
    print(Name)

    Year = soup_name.find('a').contents[0].strip()
    print(Year)

    pgRating =  soup.find('div',class_="subtext").contents[0].strip()
    print(pgRating)

    # soup = BeautifulSoup(plain_text)
    soupTry = (soup.find('div',id="titleDetails"))
    soupTry = soupTry.find('time')
    Length = soupTry.text.strip()
    print(Length)


    soupGenre = soup.find('div',id = 'titleStoryLine')
    soupGenre = soupGenre.findAll('div',class_="see-more inline canwrap")
    soupGenre = soupGenre[1].findAll('a')
    Genres = soupGenre[0].text.strip()
    for i in range(1,len(soupGenre)):
        Genres+=", "+soupGenre[i].text.strip()
    print(Genres)    


    soupDirector = soup.find('div',class_ = 'credit_summary_item')
    soupDirector = soupDirector.find('a') # first elem is director
    Director = soupDirector.text.strip()
    print(Director)    

    soupStars = soup.findAll('div',class_ = 'credit_summary_item')[2].findAll('a') 
    Stars = soupStars[0].text.strip()
    for i in range(1,len(soupStars)-1): #-1 to avoid "cast & crew" 
        Stars+=", "+soupStars[i].text.strip()
    print(Stars) 



    IMDBRating =  soup.find('span',itemprop = 'ratingValue').text.strip()    
    print(IMDBRating)

    NumberOfVotes = soup.find('span',itemprop = 'ratingCount').text.strip()    
    print(NumberOfVotes)


    soupGross = soup.find('div',id = 'titleDetails').findAll('h3', class_ ="subheading")
    Gross = None      

    if (len(soupGross) > 2 ):
        soupGross = soup.find('div',id = 'titleDetails').findAll('div',class_='txt-block')
        if(len(soupGross) >= 9 and soupGross[8].text.strip().startswith('Gross') == True):
            Gross= soupGross[8].text.strip()
            Gross = Gross.split(":")[1].strip() #only digits 
            
    print(Gross) 
    
    FullText = soup.find('div',class_="summary_text").text.strip()
    print(FullText)
    
    movie = Movie(Name,Year,pgRating,Length,Genres,Director,Stars,IMDBRating,NumberOfVotes,Gross,FullText)
    return movie


# url = "https://www.imdb.com/title/tt0770828/"

In [ ]:
def spider(url, base_url, Movies, sleep = 0, timeout = 5):
    
    time.sleep(sleep)
    source_code = requests.get(url, timeout = timeout)
    if source_code.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, source_code.status_code))   
        
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    soup = soup.find_all('h3', class_ = "lister-item-header")
    for i in soup:
        for link in i.find_all('a', href = True):
            href = base_url + link.get("href")
            Movies.append(ParseUrlToMovie(href))
            print(len(Movies))
            
# base_url = 'https://www.imdb.com'
# url = 'https://www.imdb.com/search/keyword/?keywords=time-travel&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=bdc91cb7-0144-4906-b072-b45760c8aa67&pf_rd_r=GHWVVYYYYZEFBZMW8Q4A&pf_rd_s=right-1&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_kw_17'

# spider(url = url, base_url = base_url)

In [ ]:
MOVIES_PER_PAGE = 50
REQUIRED_PAGES_NEM = 300
Movies = []
def pageCrawler(base_url,sleep = 0, timeout = 5):
    for pageNum in range(1,1+(int)(REQUIRED_PAGES_NEM/MOVIES_PER_PAGE)):
        url = "https://www.imdb.com/search/keyword/?keywords=time-travel&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=bdc91cb7-0144-4906-b072-b45760c8aa67&pf_rd_r=GHWVVYYYYZEFBZMW8Q4A&pf_rd_s=right-1&pf_rd_t=15051&pf_rd_i=genre&ref_=kw_nxt&sort=moviemeter,asc&mode=detail&page="+'pageNum'+'&title_type=movie'
        spider(url,base_url,Movies)
        
base_url = 'https://www.imdb.com'
# url = "https://www.imdb.com/search/keyword/?keywords=time-travel&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=bdc91cb7-0144-4906-b072-b45760c8aa67&pf_rd_r=FSTKKRQGAJFY7J802D0P&pf_rd_s=right-1&pf_rd_t=15051&pf_rd_i=genre&ref_=kw_nxt&sort=moviemeter,asc&mode=detail&page=2&title_type=movie"

pageCrawler(base_url=base_url)

200
Zack Snyder's Justice League
2021
R
242 min
Action, Adventure, Fantasy, Sci-Fi
Zack Snyder
Henry Cavill, Ben Affleck, Gal Gadot
8.1
267,934
None
Determined to ensure Superman's ultimate sacrifice was not in vain, Bruce Wayne aligns forces with Diana Prince with plans to recruit a team of metahumans to protect the world from an approaching threat of catastrophic proportions.
1
200
Palm Springs
2020
R
90 min
Comedy, Fantasy, Mystery, Romance
Max Barbakow
Andy Samberg, Cristin Milioti, J.K. Simmons
7.4
102,449
None
Stuck in a time loop, two wedding guests develop a budding romance while living the same day over and over again.
2
200
Tenet
2020
PG-13
150 min
Action, Sci-Fi, Thriller
Christopher Nolan
John David Washington, Robert Pattinson, Elizabeth Debicki
7.4
333,326
$58,456,624
Armed with only one word, Tenet, and fighting for the survival of the entire world, a Protagonist journeys through a twilight world of international espionage on a mission that will unfold in something beyon

200
The Devil's Advocate
1997
R
144 min
Drama, Mystery, Thriller
Taylor Hackford
Keanu Reeves, Al Pacino, Charlize Theron
7.5
336,886
$60,944,660
An exceptionally adept Florida lawyer is offered a job at a high-end New York City law firm with a high-end boss - the biggest opportunity of his career to date.
27
200
Predestination
2014
R
97 min
Action, Drama, Sci-Fi, Thriller
Michael Spierig
Ethan Hawke, Sarah Snook, Noah Taylor
7.5
256,898
$68,372
For his final assignment, a top temporal agent must pursue the one criminal that has eluded him throughout time. The chase turns into a unique, surprising and mind-bending exploration of love, fate, identity and time travel taboos.
28
200
Hot Tub Time Machine
2010
R
101 min
Comedy, Sci-Fi
Steve Pink
John Cusack, Rob Corddry, Craig Robinson
6.4
163,408
$50,287,556
A malfunctioning time machine at a ski resort takes a man back to 1986 with his two friends and nephew, where they must relive a fateful night and not change anything to make sure the 

200
Avengers: Endgame
2019
PG-13
181 min
Action, Adventure, Drama, Sci-Fi
Anthony Russo
Robert Downey Jr., Chris Evans, Mark Ruffalo
8.4
859,349
$858,373,000
After the devastating events of Avengers: Infinity War (2018), the universe is in ruins. With the help of remaining allies, the Avengers assemble once more in order to reverse Thanos' actions and restore balance to the universe.
54
200
Synchronic
2019
R
102 min
Drama, Sci-Fi, Thriller
Justin Benson
Anthony Mackie, Jamie Dornan, Katie Aselton
6.2
14,860
None
Two New Orleans paramedics' lives are ripped apart after they encounter a series of horrific deaths linked to a designer drug with bizarre, otherworldly effects.
55
200
Boss Level
2021

94 min
Action, Mystery, Sci-Fi, Thriller
Joe Carnahan
Frank Grillo, Mel Gibson, Naomi Watts
6.9
22,197
None
A retired Special Forces officer is trapped in a never-ending time loop on the day of his death.
56
200
Interstellar
2014
PG-13
169 min
Adventure, Drama, Sci-Fi
Christopher Nolan
Matthew M

200
Galaxy Quest
1999
PG
102 min
Adventure, Comedy, Sci-Fi
Dean Parisot
Tim Allen, Sigourney Weaver, Alan Rickman
7.3
151,742
None
The alumni cast of a space opera television series have to play their roles as the real thing when an alien race needs their help. However, they also have to defend both Earth and the alien race from a reptilian warlord.
80
200
X-Men: Days of Future Past
2014
PG-13
132 min
Action, Adventure, Sci-Fi, Thriller
Bryan Singer
Patrick Stewart, Ian McKellen, Hugh Jackman
7.9
666,798
$233,921,534
The X-Men send Wolverine to the past in a desperate effort to change history and prevent an event that results in doom for both humans and mutants.
81
200
Minority Report
2002
PG
145 min
Action, Crime, Mystery, Sci-Fi, Thriller
Steven Spielberg
Tom Cruise, Colin Farrell, Samantha Morton
7.6
513,509
$132,072,926
In a future where a special police unit is able to arrest murderers before they commit their crimes, an officer from that unit is himself accused of a future murder

200
Interstellar
2014
PG-13
169 min
Adventure, Drama, Sci-Fi
Christopher Nolan
Matthew McConaughey, Anne Hathaway, Jessica Chastain
8.6
1,546,368
$188,020,017
A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.
107
200
Batman v Superman: Dawn of Justice
2016
PG-13
152 min
Action, Adventure, Sci-Fi
Zack Snyder
Ben Affleck, Henry Cavill, Amy Adams
6.4
646,862
$330,360,194
Fearing that the actions of Superman are left unchecked, Batman takes on the Man of Steel, while the world wrestles with what kind of a hero it really needs.
108
200
Back to the Future
1985

116 min
Adventure, Comedy, Sci-Fi
Robert Zemeckis
Michael J. Fox, Christopher Lloyd, Lea Thompson
8.5
1,075,755
$211,406,762
Marty McFly, a 17-year-old high school student, is accidentally sent thirty years into the past in a time-traveling DeLorean invented by his close friend, the eccentric scientist Doc Brown.
109
200
Deadpool 2
2018
R
119 min
Action, Adventure, Comedy, Sci-Fi
David

...

200
Miss Peregrine's Home for Peculiar Children
2016
PG-13
127 min
Adventure, Drama, Family, Fantasy, Thriller
Tim Burton
Eva Green, Asa Butterfield, Samuel L. Jackson
6.7
157,948
$87,242,834
When Jacob (Asa Butterfield) discovers clues to a mystery that stretches across time, he finds Miss Peregrine's Home for Peculiar Children. But the danger deepens after he gets to know the residents and learns about their special powers.
186
200
Star Trek
2009
PG-13
127 min
Action, Adventure, Sci-Fi
J.J. Abrams
Chris Pine, Zachary Quinto, Simon Pegg
7.9
580,235
$257,730,019
The brash James T. Kirk tries to live up to his father's legacy with Mr. Spock keeping him in check as a vengeful Romulan from the future creates black holes to destroy the Federation one planet at a time.
187
200
Idiocracy
2006
R
84 min
Adventure, Comedy, Sci-Fi, Thriller
Mike Judge
Luke Wilson, Maya Rudolph, Dax Shepard
6.6
148,092
None
Private Joe Bauers, a decisively average American, is selected as a guinea pig for a top-s

200
About Time
2013
R
123 min
Comedy, Drama, Fantasy, Romance, Sci-Fi
Richard Curtis
Domhnall Gleeson, Rachel McAdams, Bill Nighy
7.8
311,245
$15,322,921
At the age of 21, Tim discovers he can travel in time and change what happens and has happened in his own life. His decision to make his world a better place by getting a girlfriend turns out not to be as easy as you might think.
212
200
Bill & Ted Face the Music
2020
PG-13
91 min
Adventure, Comedy, Music, Sci-Fi
Dean Parisot
Keanu Reeves, Alex Winter, Kristen Schaal
6.0
31,941
None
Once told they'd save the universe during a time-traveling adventure, 2 would-be rockers from San Dimas, California find themselves as middle-aged dads still trying to crank out a hit song and fulfill their destiny.
213
200
Doctor Strange
2016
PG-13
115 min
Action, Adventure, Fantasy, Sci-Fi
Scott Derrickson
Benedict Cumberbatch, Chiwetel Ejiofor, Rachel McAdams
7.5
613,151
$232,641,920
While on a journey of physical and spiritual healing, a brilliant neur

200
Idiocracy
2006
R
84 min
Adventure, Comedy, Sci-Fi, Thriller
Mike Judge
Luke Wilson, Maya Rudolph, Dax Shepard
6.6
148,092
None
Private Joe Bauers, a decisively average American, is selected as a guinea pig for a top-secret hibernation program but is forgotten, awakening to a future so incredibly moronic he's easily the most intelligent person alive.
238
200
Jumanji
1995
PG
104 min
Adventure, Comedy, Family, Fantasy
Joe Johnston
Robin Williams, Kirsten Dunst, Bonnie Hunt
7.0
313,422
$100,499,940
When two kids find and play a magical board game, they release a man trapped in it for decades - and a host of dangers that can only be stopped by finishing the game.
239
200
Austin Powers: International Man of Mystery
1997
PG-13
94 min
Adventure, Comedy
Jay Roach
Mike Myers, Elizabeth Hurley, Michael York
7.0
223,751
$53,883,989
A world-class playboy and part-time secret agent from the 1960s emerges after thirty years in a cryogenic state to battle with his nemesis Dr. Evil.
240
200
The Map

200
Doctor Strange
2016
PG-13
115 min
Action, Adventure, Fantasy, Sci-Fi
Scott Derrickson
Benedict Cumberbatch, Chiwetel Ejiofor, Rachel McAdams
7.5
613,151
$232,641,920
While on a journey of physical and spiritual healing, a brilliant neurosurgeon is drawn into the world of the mystic arts.
264
200
Spider-Man: Into the Spider-Verse
2018
PG
117 min
Animation, Action, Adventure, Family, Sci-Fi
Bob Persichetti
Shameik Moore, Jake Johnson, Hailee Steinfeld
8.4
393,754
$190,241,310
Teen Miles Morales becomes the Spider-Man of his universe, and must join with five spider-powered individuals from other dimensions to stop a threat for all realities.
265
200
Arrival
2016
PG-13
116 min
Drama, Sci-Fi
Denis Villeneuve
Amy Adams, Jeremy Renner, Forest Whitaker
7.9
606,944
$100,546,139
A linguist works with the military to communicate with alien lifeforms after twelve mysterious spacecraft appear around the world.
266
200
Dark Shadows
2012
PG-13
113 min
Comedy, Fantasy, Horror
Tim Burton
Johnny Dep

200
Austin Powers: International Man of Mystery
1997
PG-13
94 min
Adventure, Comedy
Jay Roach
Mike Myers, Elizabeth Hurley, Michael York
7.0
223,751
$53,883,989
A world-class playboy and part-time secret agent from the 1960s emerges after thirty years in a cryogenic state to battle with his nemesis Dr. Evil.
290
200
The Map of Tiny Perfect Things
2021
PG-13
98 min
Comedy, Fantasy, Romance
Ian Samuels
Kathryn Newton, Kyle Allen, Jermaine Harris
6.8
11,775
None
Two teens who live the same day repeatedly, enabling them to create the titular map.
291
200
Back to the Future Part II
1989

108 min
Adventure, Comedy, Sci-Fi
Robert Zemeckis
Michael J. Fox, Christopher Lloyd, Lea Thompson
7.8
487,915
$119,000,002
After visiting 2015, Marty McFly must repeat his visit to 1955 to prevent disastrous changes to 1985...without interfering with his first trip.
292
200
2067
2020

114 min
Sci-Fi
Seth Larney
Aaron Glenane, Kodi Smit-McPhee, Ryan Kwanten
4.9
5,768
None
One man's journey to the future to s

In [ ]:
import json
# print(len(Movies))
json_string = json.dumps([m.__dict__ for m in Movies])
parsed = json.loads(json_string)
json.dumps(parsed, indent=4)
with open('data111.json', 'w') as outfile:
     json.dump([m.__dict__ for m in Movies], outfile, indent=4)